In [1]:
%matplotlib inline
''' Python Tools '''
import numpy as np
np.set_printoptions(suppress=True) # Changes print functionality
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

'''PyTorch Functionality'''
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

'''Landscape Class'''
from Protein_Landscape.landscape_class import Protein_Landscape

'''Dataset Management'''
Datasets = 'Data/'


In [2]:
test = Protein_Landscape(csv_path=Datasets+"Small_NK.csv",gen_graph=True)


Building Protein Graph for entire dataset


100%|██████████| 1000/1000 [00:00<00:00, 66687.40it/s]

Calculating the number of extrema

        Protein Landscape class
            Number of Sequences : 1000
            Max Distance        : 3
            Number of Distances : 3
            Seed Sequence       : AAA
                Modified positions are shown in green
            Number of minima : 1
            Number of maxima : 1
            Normalized Extrema Ruggedness : 0.002
            R/S Ruggedness : 17.319433978287012
        


In [6]:
len(test.indexing(positions=[1,2],distances=2,percentage=0.3))

24

In [3]:
test[np.array([63,87])]

array([['AHE', 0.2117072904890552],
       ['AKI', 0.5108756599934917]], dtype=object)

In [6]:
idxs = test.indexing(positions=[1],percentage=1)

idxs == test.indexing(positions=[1])

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

array([['AAC', 0.3663419495182938],
       ['AAD', 0.6457100193331815],
       ['AAE', 0.4342208225137681],
       ['AAF', 0.3746929246583692]], dtype=object)

In [4]:
test.data[[1,2,3]]

array([['AAC', 0.3663419495182938],
       ['AAD', 0.6457100193331815],
       ['AAE', 0.4342208225137681]], dtype=object)

In [16]:
idx = [idx for idx in idxs][1]
test.graph[idx]

{'tokenized': (0, 1, 2),
 'string': 'ACD',
 'fitness': 0.6989436073974228,
 'neighbours': array([  2,  10,  11,  13,  14,  15,  16,  17,  18,  19,  22,  32,  42,
         52,  62,  72,  82,  92, 112, 212, 312, 412, 512, 612, 712, 812,
        912])}